In [2]:
import pandas as pd
import sqlite3
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder


db_path = 'data.db'
connection = sqlite3.connect(db_path)

queryTrain = "SELECT * FROM teams WHERE year < 10 ORDER BY year"
train_data = pd.read_sql_query(queryTrain, connection)

queryTest = "SELECT * FROM teams WHERE year = 10"
test_data = pd.read_sql_query(queryTest, connection)

# Define your features (X) and target (y) for training data
X_train = train_data.drop(columns=['playoff', 'lgID', 'tmID', 'franchID','confID','divID', 'seeded', 'semis','finals','name', 'firstRound', 'arena', 'rank'])  # Exclude non-feature columns
y_train = train_data['playoff']  # Your target variable

# Define your features (X) and target (y) for test data
X_test = test_data.drop(columns=['playoff', 'lgID', 'tmID', 'franchID','confID','divID', 'seeded', 'semis','finals','name', 'firstRound', 'arena', 'rank'])  # Exclude non-feature columns
y_test = test_data['playoff']  # Your target variable

# Create a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=0)

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

#print(tree.export_text(clf))
""" print(X_train.columns)
print(X_train['homeL'])
 """
print(tree.export_text(clf))

# Calculate accuracy on the test data
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy: .2f}")

print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)



|--- feature_37 <= 16.50
|   |--- feature_45 <= 9.50
|   |   |--- feature_8 <= 600.50
|   |   |   |--- class: N
|   |   |--- feature_8 >  600.50
|   |   |   |--- class: Y
|   |--- feature_45 >  9.50
|   |   |--- feature_27 <= 317.50
|   |   |   |--- class: N
|   |   |--- feature_27 >  317.50
|   |   |   |--- class: Y
|--- feature_37 >  16.50
|   |--- feature_19 <= 719.50
|   |   |--- feature_45 <= 12.50
|   |   |   |--- class: Y
|   |   |--- feature_45 >  12.50
|   |   |   |--- feature_21 <= 486.50
|   |   |   |   |--- class: N
|   |   |   |--- feature_21 >  486.50
|   |   |   |   |--- class: Y
|   |--- feature_19 >  719.50
|   |   |--- feature_37 <= 19.00
|   |   |   |--- feature_18 <= 632.50
|   |   |   |   |--- class: N
|   |   |   |--- feature_18 >  632.50
|   |   |   |   |--- class: Y
|   |   |--- feature_37 >  19.00
|   |   |   |--- class: Y

Accuracy:  0.69
              precision    recall  f1-score   support

           N       0.56      1.00      0.71         5
           Y  